# Mesogens with NP | Temperature Ramps

## Temperature: 9.0 -> 9.2 | CPU Run

### Date: 08/01/2020 | System P = 1.8, Expected value of $T_c$ : 7.09 

In [1]:
#-----Import essential packages.

from __future__ import division
import hoomd
import hoomd.md

In [2]:
#-----Define important variables.

p_max = 2.3;
t_0   = 9.0; 
t_1   = 9.2;
steps_ramp = 5e4;

In [3]:
#-----Declare the file from wich we'll extract the configuration of the system.

ramp_file = "T_CM&NP_" + str(t_0) + "_P_" + str(p_max) + "_ramp.gsd"

In [4]:
#-----Declare a simulation context.

hoomd.context.initialize("--mode=cpu", memory_traceback = True);

#-----Initialize from a .gsd file.

system = hoomd.init.read_gsd(ramp_file, frame = -1);

#-----Define each mesogen in the local frame of reference of each rigid body.

rigid = hoomd.md.constrain.rigid();
rigid.set_param('M', 
                types = ['A']*8,
                positions = [(-4,0,0),(-3,0,0),(-2,0,0),(-1,0,0),
                           (1,0,0),(2,0,0),(3,0,0),(4,0,0)]);

#-----Create the mesogens.

rigid.create_bodies();

#-----Define the potential energy.

nl = hoomd.md.nlist.tree(r_buff = 3.5);

lj = hoomd.md.pair.lj(r_cut = 3.5, nlist = nl)
lj.set_params(mode = 'shift')

#-----Declare interactions.

lj.pair_coeff.set('NP','NP', epsilon = 1.0, sigma = 5.0);
lj.pair_coeff.set('M' ,'M' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('A' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('M' ,'A' , epsilon = 1.0, sigma = 1.0);
lj.pair_coeff.set('NP','M' , epsilon = 1.0, sigma = 3.0);
lj.pair_coeff.set('NP','A' , epsilon = 1.0, sigma = 3.0);

#-----Select an standard integrator.

hoomd.md.integrate.mode_standard(dt = 0.005);

#------Define some groups and make their union.

nanoparticles = hoomd.group.type(name = 'Nano_Particles', type = 'NP');
mesogens = hoomd.group.rigid_center();
groupNP_mes = hoomd.group.union(name = "NP_Mes", a = nanoparticles, b = mesogens);

#----Integrate using an NPT integrator.

temp = hoomd.variant.linear_interp(points = [(0,t_0), (steps_ramp, t_1)]);

npt = hoomd.md.integrate.npt(group = groupNP_mes, kT = temp, tau = 1.5, tauP = 2.0, P = p_max);
npt.randomize_velocities(seed=42)

#-----Save .log y .gsd data.

log_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.log"
gsd_file = "T_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd" 
meso_gsd_file = "T_CM&NP_" + str(t_1) + "_P_" + str(p_max) + "_ramp.gsd"

log = hoomd.analyze.log(filename = log_file,
                         quantities = ['num_particles',
                                     'ndof',
                                     'translational_ndof',
                                     'rotational_ndof',
                                     'potential_energy',
                                     'kinetic_energy',
                                     'translational_kinetic_energy',
                                     'rotational_kinetic_energy',
                                     'temperature',
                                     'pressure',
                                      'volume'],
                         period = 1e3,
                         overwrite = True);
gsd = hoomd.dump.gsd(gsd_file,
               period = 1e3,
               group = hoomd.group.all(),
               overwrite = True); 
meso_gsd = hoomd.dump.gsd(meso_gsd_file,
               period = 1e3,
               group = groupNP_mes,
               overwrite = True); 

HOOMD-blue 2.7.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 10/02/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU
notice(2): Group "all" created containing 1010 particles
notice(2): constrain.rigid(): Creating 1000 rigid bodies (adding 8000 particles)
-----
You are using tree neighbor lists. Please cite the following:
* M P Howard, J A Anderson, A Nikoubashman, S C Glotzer, and A Z
  Panagiotopoulos. "Efficient neighbor list calculation for mol

In [ ]:
#-----Run the simulation.

hoomd.run(steps_ramp)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 9010
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: yes
** starting run **
Time 00:00:10 | Step 250197 / 300000 | TPS 19.6942 | ETA 00:42:08
Time 00:00:49 | Step 250201 / 300000 | TPS 0.10218 | ETA 135:22:45
Time 00:01:24 | Step 250301 / 300000 | TPS 2.83375 | ETA 04:52:18
Time 00:01:59 | Step 250401 / 300000 | TPS 2.84581 | ETA 04:50:28
Time 00:02:34 | Step 250501 / 300000 | TPS 2.89314 | ETA 04:45:09
Time 00:03:08 | Step 250601 / 300000 | TPS 2.9478 | ETA 04:39:17
Time 00:03:42 | Step 250701 / 300000 | TPS 2.93857 | ETA 04:39:36
Time 00:04:15 | Step 250801 / 300000 | TPS 2.98722 | ETA 04:34:29
Time 00:04:48 | Step 250901 / 300000 | TPS 3.01078 | ETA 04:31:47
Time 00:05:21 | Step 251001 / 300000 | TPS 3.01934 | ETA 04:30:28
Time 00:05:55 | Step 251101 / 300000 | TPS 3.02371 | ETA 04:29:31
Time 00:06:27 | Step 251201 / 3

In [ ]:
#-----Get density.

system.get_metadata()

In [ ]:
#-----Get volume.

system.box.get_volume()